In [11]:

# refined distances calculation
import faiss
def _refine_distances(xq_ds, idx, xb_ds, I,k,metric):
        xq = xq_ds.get(idx).repeat(k, axis=0)
        xb = xb_ds.get(I.reshape(-1))
        if metric == faiss.METRIC_INNER_PRODUCT:
            return (xq * xb).sum(axis=1).reshape(I.shape)
        elif metric == faiss.METRIC_L2:
            return ((xq - xb) ** 2).sum(axis=1).reshape(I.shape)
        else:
            raise ValueError(f"metric not supported {metric}")


In [13]:
# python3 run.py --command evaluate --config config_unittest.yaml --xb coco_lang --xq goku_lang 
import sys

sys.path.append("/private/home/marialomeli/faiss_improvements/offline_ivf/")
from utils import load_config
from dataset import create_dataset_from_oivf_config
import numpy as np
class Object(object):
    pass
cfg = load_config("/private/home/marialomeli/faiss_improvements/offline_ivf" + "/config_unittest.yaml")
self = Object()
# self.root = '/scratch/gsz/seamless/oivf'
self.root = "/private/home/marialomeli/faiss_improvements/offline_ivf/tests/test_data"

self.eval_dir = self.root + "/goku_lang_in_coco_lang/eval"
self.index_factory_fn = cfg["index"]["prod"]
self.nprobe = cfg["nprobe"]["prod"]
self.k = cfg["k"]
self.metric = cfg["metric"]
I_a_ann = np.load(self.eval_dir+"/I_a_ann_OPQ4_IVF256_PQ4_np2.npy")

self.xq_ds = create_dataset_from_oivf_config(cfg,"goku_lang")
self.xb_ds = create_dataset_from_oivf_config(cfg,"coco_lang")
idx_a = np.load(self.eval_dir+"/idx_a.npy")
idx_a
I_a_ann[:10,]
#D_a_ann_refined = _refine_distances(self.xq_ds, idx_a, self.xb_ds, I_a_ann,self.k,faiss.METRIC_L2)

array([[   -1,    -1],
       [   -1,    -1],
       [18237, 29647],
       [   -1,    -1],
       [29580, 11368],
       [   -1,    -1],
       [   -1,    -1],
       [   -1,    -1],
       [ 7786,  3227],
       [10255, 20138]])

In [9]:
D_a_ann_refined

array([[0.0652544 , 0.12947224],
       [0.02952097, 0.06808341],
       [0.0934061 , 0.08538042],
       [0.06198333, 0.08373729],
       [0.05951791, 0.07040385],
       [0.03912928, 0.06485844],
       [0.09571987, 0.0804907 ],
       [0.05539767, 0.04808123],
       [0.04487   , 0.09408781],
       [0.04513822, 0.05732442],
       [0.07468422, 0.06742606],
       [0.09897658, 0.11005229],
       [0.05      , 0.07084642],
       [0.05911131, 0.05821372],
       [0.08761284, 0.08724706],
       [0.07609091, 0.07267965],
       [0.06572562, 0.09638156],
       [0.03914481, 0.0559928 ],
       [0.0671391 , 0.06793108],
       [0.07346898, 0.10121743],
       [0.05885801, 0.08078897],
       [0.03797341, 0.06807525],
       [0.08974423, 0.06393065],
       [0.0812999 , 0.07795068],
       [0.05914135, 0.05397893],
       [0.07023679, 0.08861698],
       [0.05998095, 0.07408154],
       [0.09624723, 0.06215914],
       [0.03952349, 0.0806938 ],
       [0.07509121, 0.09342497],
       [0.

In [9]:
import faiss
def _coarse_quantize(index_ivf, xq, nprobe):
    assert nprobe <= index_ivf.quantizer.ntotal
    quantizer = faiss.index_cpu_to_all_gpus(index_ivf.quantizer)
    bs = 65536
    nq = len(xq)
    q_assign = np.empty((nq, nprobe), dtype="int32")
    for i0 in range(0, nq, bs):
        i1 = min(nq, i0 + bs)
        _, q_assign_i = quantizer.search(xq[i0:i1], nprobe)
        q_assign[i0:i1] = q_assign_i
    return q_assign


In [10]:
import faiss
import sys

sys.path.append("/private/home/marialomeli/faiss_improvements/offline_ivf/")
from utils import load_config
from dataset import create_dataset_from_oivf_config
import numpy as np
class Object(object):
    pass
class Knn():
    def __init__(self):
        ngpu=faiss.get_num_gpus()
        self.all_gpu_resources = [faiss.StandardGpuResources() for _ in range(ngpu)]
    def _knn_function(self,xq, xb, k, metric, thread_id=None):
        """
        knn GPU method
        """
        try:
            # print(f"knn_function: {xq.shape}, {xb.shape}")
            if xq.shape[0] > 131_072 or xb.shape[0] > 1_048_576:
                return self._knn_safe(thread_id, xq, xb, k, metric=metric)
            else:
                return faiss.knn_gpu(
                    self.all_gpu_resources[thread_id],
                    xq,
                    xb,
                    k,
                    metric=metric,
                    device=thread_id,
                )
        except:
            print(f"knn_function failed: {xq.shape}, {xb.shape}")
            raise

    def _knn_safe(self,thread_id, xq, xb, k, metric):
            print(f"knn_safe: {xq.shape}, {xb.shape}")
            nq, d = xq.shape
            keep_max = faiss.is_similarity_metric(metric)
            rh = faiss.ResultHeap(nq, k, keep_max=keep_max)
            index = faiss.IndexFlat(d, metric)
            index = faiss.index_cpu_to_gpu(self.all_gpu_resources[thread_id], thread_id, index)
            bs = 1_048_576
            for xbi in range(0, xb.shape[0], bs):
                xbs = xb[xbi : xbi + bs]
                index.add(xbs)
                D, I = index.search(xq, k)
                I += xbi
                rh.add_result(D, I)
                index.reset()
            rh.finalize()
            return rh.D, rh.I
        
        
cfg = load_config("/private/home/marialomeli/faiss_improvements/offline_ivf" + "/config_sonar.yaml")
self = Object()
self.root = '/checkpoint/gsz/seamless/sonar/glg'
self.eval_dir = self.root + "/glg_in_glg/knn"
self.index_factory_fn = cfg["index"]["prod"]
self.nprobe = cfg["nprobe"]["prod"]
self.k = cfg["k"]
self.metric = faiss.METRIC_L2
index=faiss.read_index("/checkpoint/marialomeli/offline_faiss/seamless/sonar/glg/IVF32768_PQ512.shard_0",faiss.IO_FLAG_MMAP | faiss.IO_FLAG_READ_ONLY)
self.xb_ds = create_dataset_from_oivf_config(cfg,"glg")
self.xq_ds = self.xb_ds

knn=Knn()
self._knn_function=knn._knn_function(
    np.zeros((10, 10), dtype="float32"),
    np.zeros((10, 10), dtype="float32"),
    self.k,
    metric=self.metric,
    thread_id=0,
)

In [6]:
self.nprobe[0]

128

In [11]:
from faiss.contrib.ivf_tools import big_batch_search
dq_iterator=self.xq_ds.iterate(0, 10_000, np.float32) 
xq_i=next(dq_iterator)
D,I=index.search(xq_i,self.k)
index_ivf = faiss.downcast_index(faiss.extract_index_ivf(index))
q_assign = _coarse_quantize(index_ivf, xq_i, self.nprobe[0])
D_bbs, I_bbs = big_batch_search(
                        index_ivf,
                        xq_i,
                        self.k,
                        verbose=10,
                        method="knn_function",
                        knn=knn._knn_function,
                        threaded=faiss.get_num_gpus() * 8,
                        use_float16=True,
                        prefetch_threads=faiss.get_num_gpus(),
                        computation_threads=faiss.get_num_gpus(),
                        q_assign=q_assign,
                    )
#1 big batch of 50,000,000 vectors check results

processing: encf.000.glg...
memory: queries 40960000 assign 40000 result 1920000 total 42920000 = 0.040 GiB
  number of -1s: 0
bucket sort: 0.015 s


In [17]:
import glob
path="/checkpoint/marialomeli/offline_faiss/seamless/sonar/glg_in_glg/knn"
I_files=glob.glob(path+"/I*")
print(sorted(I_files))

['/checkpoint/marialomeli/offline_faiss/seamless/sonar/glg_in_glg/knn/I0000000000_IVF32768_PQ512_np128.npy', '/checkpoint/marialomeli/offline_faiss/seamless/sonar/glg_in_glg/knn/I0020000000_IVF32768_PQ512_np128.npy', '/checkpoint/marialomeli/offline_faiss/seamless/sonar/glg_in_glg/knn/I0040000000_IVF32768_PQ512_np128.npy', '/checkpoint/marialomeli/offline_faiss/seamless/sonar/glg_in_glg/knn/I0060000000_IVF32768_PQ512_np128.npy', '/checkpoint/marialomeli/offline_faiss/seamless/sonar/glg_in_glg/knn/I0080000000_IVF32768_PQ512_np128.npy', '/checkpoint/marialomeli/offline_faiss/seamless/sonar/glg_in_glg/knn/I0100000000_IVF32768_PQ512_np128.npy', '/checkpoint/marialomeli/offline_faiss/seamless/sonar/glg_in_glg/knn/I0120000000_IVF32768_PQ512_np128.npy', '/checkpoint/marialomeli/offline_faiss/seamless/sonar/glg_in_glg/knn/I0140000000_IVF32768_PQ512_np128.npy', '/checkpoint/marialomeli/offline_faiss/seamless/sonar/glg_in_glg/knn/I0160000000_IVF32768_PQ512_np128.npy', '/checkpoint/marialomeli/of

In [ ]:
#batch_size>file_size
num_split_files_per_batch = batch_size//file_size
if batch_size%file_size!=0:
    num_split_files_per_batch +=1
split_positions = [file_size0,file_size0+file_size1]
I_all = np.array_split(I,split_positions) #splits in 3, file_size0,filesize1 and batch_size-file_size if remainder
# save remainder and handle the case when we need to stack it, then the split_positions are offset by this :(

In [57]:
import os
print("split_npy_files")
def split_merge_files(I_files,output_file_sizes,xq_bs,k):

    num_input_files = len(I_files)
    total_input_size = num_input_files*xq_bs #only if there is no leftover
    total_output_size = sum(output_file_sizes)

    if total_input_size > total_output_size:
        I_last_file = np.load(I_files[num_input_files-1])
        remainder = total_output_size - (num_input_files-1)*xq_bs
        print("assertion",remainder == I_last_file.shape[0])
        assert remainder == I_last_file.shape[0],f"the total input and output sizes do not match:{total_input_size-xq_bs+I_last_file.shape[0]},{total_output_size}"
    else:
        assert total_input_size == total_output_size, f"the total input and output sizes do not match:{total_input_size},{total_output_size}"
    # D_files = sorted([f"{self.knn_dir}/D_approx{(i):010}_{self.knn_output_file_suffix}" for i in range(0, self.xq_ds.size, self.xq_bs)])
    output_files = []

    current_input_file_index = 0
    current_input_file_offset = 0
    for jj, current_output_file_size in enumerate(output_file_sizes):
        output_file = np.empty((0, k))
        while output_file.shape[0] < current_output_file_size:
            still_need = current_output_file_size - output_file.shape[0]
            I_current_file = np.load(I_files[current_input_file_index])
            max_we_can_read_from_current_input = min(
                I_current_file.shape[0], current_input_file_offset + still_need
            )
            input_chunk = I_current_file[current_input_file_offset:max_we_can_read_from_current_input,]
            output_file = np.vstack([output_file, input_chunk])
            current_input_file_offset = max_we_can_read_from_current_input
            if current_input_file_offset == I_current_file.shape[0]:
                current_input_file_index += 1
                current_input_file_offset = 0
        output_files.append(output_file)
        np.save(f"I_output_split_{jj}", output_file)
    return output_files

split_npy_files


In [58]:
import numpy as np
import glob
k = 10
output_file_sizes = [123, 543, 620, 149]
print(sum(output_file_sizes))
batch_size = 1000
input_files = [np.random.rand(batch_size, k), np.random.rand(435, k)]
for jj,file in enumerate(input_files):
    np.save(f"I_input{(jj):010}",file)
wkdir = os.getcwd()
I_files = sorted(glob.glob(wkdir + "/I_input*"))
print(I_files) 
#I_files = [np.load(file) for ]
out = split_merge_files(I_files,output_file_sizes,batch_size,k)


1435
['/private/home/marialomeli/faiss_improvements/offline_ivf/notebooks/I_input0000000000.npy', '/private/home/marialomeli/faiss_improvements/offline_ivf/notebooks/I_input0000000001.npy']
assertion True
